In [1]:
from __future__ import absolute_import
from __future__ import print_function
"""
https://github.com/tdeboissiere/DeepLearningImplementations/tree/master/DenseNet
"""
import numpy as np
import random
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt

import keras.backend as K
from keras.initializers import RandomNormal
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Input,concatenate,Concatenate
from keras.layers import normalization, BatchNormalization, Lambda
from keras.layers import Flatten, Conv2D
from keras.regularizers import l2
from keras.optimizers import RMSprop,Adam
from keras.utils import np_utils
from keras.models import Model,Sequential
from keras.wrappers.scikit_learn import KerasClassifier
import densenet
from keras.datasets import mnist
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

"""                                           ------Configs------                                         """
is_flipped_labels = False
is_using_densenet = False
is_using_distance = False
is_using_contrastive_loss = True

print("                                          ------Configs------                                         ")
print(" is_flipped_labels : ", is_flipped_labels)
print(" is_using_densenet : ", is_using_densenet)
print(" is_using_distance : ", is_using_distance)
print(" is_using_contrastive_loss : ", is_using_contrastive_loss)
num_classes = 2
epochs = 5
seed = 7
np.random.seed(seed)
batch_size = 32

def load_data():
    f = h5py.File('matchedImagesSplitClasses-2017-02-24-17-39-44-96-96-split-val0.15-tr0.7-tst0.15.hdf5','r')
    ln_training = 39840
    X_train = f['X_train'].value
    X_train_resize = X_train[0:ln_training,:,:,:]
    #print(X_train_resize.shape)
    y_train = f['y_train'].value
    y_train_resize = y_train[0:ln_training,]
    y_train_categorical = np_utils.to_categorical(y_train_resize, 2)
    ln_validation = 7440
    X_val = f['X_val'].value
    X_val_resize = X_val[0:ln_validation,:,:,:]
    y_val = f['y_val'].value
    y_val_reshaped = y_val[0:ln_validation]
    y_val_categorical = np_utils.to_categorical(y_val_reshaped, 2)
    #print(X_val_reshaped.shape)
    #print(y_val_categorical.shape)

    tr_pair1 = X_train_resize[:,0,:,:]
    tr_pair1_reshaped = tr_pair1.reshape(X_train_resize.shape[0],1,X_train_resize.shape[2],X_train_resize.shape[3])
    tr_pair2 = X_train_resize[:,1,:,:]
    tr_pair2_reshaped = tr_pair2.reshape(X_train_resize.shape[0],1,X_train_resize.shape[2],X_train_resize.shape[3])
    #print (tr_pair1_reshaped.shape,tr_pair2_reshaped.shape)
    #print (y_train_resize.shape)

    te_pair1 = X_val_resize[:,0,:,:]
    te_pair1_reshaped = te_pair1.reshape(X_val_resize.shape[0],1,X_val_resize.shape[2],X_val_resize.shape[3])
    te_pair2 = X_val_resize[:,1,:,:]
    te_pair2_reshaped = te_pair2.reshape(X_val_resize.shape[0],1,X_val_resize.shape[2],X_val_resize.shape[3])
    #print (te_pair1_reshaped.shape,te_pair2_reshaped.shape)
    
    #return (tr_pair1_reshaped,tr_pair1_reshaped,y_train_categorical),(te_pair1_reshaped,te_pair2_reshaped,y_val_categorical),y_val

    if is_using_densenet:
        return (tr_pair1_reshaped,tr_pair2_reshaped,y_train),(te_pair1_reshaped,te_pair2_reshaped,y_val)
    else:
        return (X_train,y_train),(X_val,y_val)

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    loss = K.mean(y_true * K.square(y_pred) +
                  (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))
    return loss


def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)


def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

def create_base_network_simple(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

def create_densenet_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''    
    #default values
    nb_classes = 2
    depth =  7 #40 #3N+4
    nb_filter = 16 #16 #increasing this worsens the result
    nb_dense_block = 1 #3
    growth_rate = 12
    dropout_rate = 0.1
    learning_rate = 1E-3
    weight_decay = 1E-4

    net = densenet.DenseNet(nb_classes,
                  input_shape,
                  depth,
                  nb_dense_block,
                  growth_rate,
                  nb_filter,
                  dropout_rate=dropout_rate,
                  weight_decay=weight_decay)
    
    return net

def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

def plot_auc(fpr,tpr):
    plt.figure()
    plt.plot(fpr, tpr,'m-')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.show()

if is_using_densenet:
    (x_train1,x_train2, y_train_real), (x_test1,x_test2, y_test_real) = load_data() #todo change to one array
    input_shape = (1,96,96) #x_train_real.shape[2:]
else:
    (x_train_real, y_train_real), (x_test_real, y_test_real) = load_data()
    input_shape = x_train_real.shape[2:] #(1,96,96)
    
if is_flipped_labels:
    y_train_real = 1 - y_train_real
    y_test_real = 1 - y_test_real

# network definition
if is_using_densenet:
    base_network = create_densenet_base_network(input_shape)
else:
    base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)


if is_using_distance:
    distance = Lambda(euclidean_distance,
                      output_shape=eucl_dist_output_shape)([processed_a, processed_b])
    model = Model([input_a, input_b], distance)
else:
    combined_features = concatenate([processed_a, processed_b], name = 'merge_features')
    combined_features = Dense(64)(combined_features)
    combined_features = BatchNormalization()(combined_features)
    combined_features = Activation('relu')(combined_features)
    combined_features = Dense(1, activation = 'sigmoid')(combined_features)
    
    model = Model(inputs = [input_a, input_b], outputs = [combined_features], name = 'Similarity_Model')

model.summary()
# train
rms = RMSprop()
if is_using_contrastive_loss:
    model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
else:
    model.compile(loss='binary_crossentropy', optimizer=rms, metrics=['accuracy'])

if is_using_densenet:
    model.fit([x_train1,x_train2],y_train_real,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=([x_test1,x_test2], y_test_real),
              verbose=1)    
    
else:
    model.fit([x_train_real[:, 0],x_train_real[:, 1]], y_train_real,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=([x_test_real[:, 0],x_test_real[:, 1]], y_test_real),
          verbose=1)    
    
if is_using_contrastive_loss:
    if is_using_densenet:
        y_pred = model.predict([x_train1,x_train2])
        tr_acc = compute_accuracy(y_train_real, y_pred)
        y_pred = model.predict([x_test1,x_test2])
        te_acc = compute_accuracy(y_test_real, y_pred)        
        
    else:
        y_pred = model.predict([x_train_real[:, 0],x_train_real[:, 1]])
        tr_acc = compute_accuracy(y_train_real, y_pred)
        y_pred = model.predict([x_test_real[:, 0],x_test_real[:, 1]])
        te_acc = compute_accuracy(y_test_real, y_pred)

    print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
    print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))
    if is_using_densenet:
        roc_auc_score = metrics.roc_auc_score(y_test_real, 1-y_pred)
    else:
        roc_auc_score = metrics.roc_auc_score(y_test_real, y_pred)
    print("roc_auc_score:  %0.2f" % roc_auc_score)
else:
    if is_using_densenet:        
        y_pred = model.predict([x_test1,x_test2])
        
    else:
        y_pred = model.predict([x_test_real[:, 0],x_test_real[:, 1]])
    roc_auc_score = metrics.roc_auc_score(y_test_real, y_pred)
    print("roc_auc_score:  %0.2f" % roc_auc_score)

/home/arka/anaconda2/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


                                          ------Configs------                                         
 is_flipped_labels :  False
 is_using_densenet :  True
 is_using_distance :  False
 is_using_contrastive_loss :  True
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1, 96, 96)    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1, 96, 96)    0                                            
__________________________________________________________________________________________________
DenseNet (Model)                (None, 2688)         15728       input_2[0][0]                    
                                                                 input_3[0][0]        

In [ ]:
"""Results: 
Config: 

is_flipped_labels = True
is_using_densenet = True
is_using_distance = True
is_using_contrastive_loss = True

* Accuracy on training set: 51.95%
* Accuracy on test set: 55.51%
roc_auc_score:  0.54
---------------------------------------
Config:
    
is_flipped_labels = False
is_using_densenet = False
is_using_distance = False
is_using_contrastive_loss = False

roc_auc_score:  0.83
---------------------------------------
Config:
    
is_flipped_labels = False
is_using_densenet = False
is_using_distance = False
is_using_contrastive_loss = True

* Accuracy on training set: 97.98%
* Accuracy on test set: 77.08%
roc_auc_score:  0.90
    
---------------------------------------
Config:
    
is_flipped_labels = True
is_using_densenet = False
is_using_distance = False
is_using_contrastive_loss = True

* Accuracy on training set: 97.91%
* Accuracy on test set: 76.60%
roc_auc_score:  0.88
    
---------------------------------------
Config:
    
is_flipped_labels :  False
is_using_densenet :  True
is_using_distance :  True
is_using_contrastive_loss :  True

* Accuracy on training set: 80.19%
* Accuracy on test set: 63.16%
roc_auc_score:  0.64
---------------------------------------
Config:

is_flipped_labels :  False
is_using_densenet :  False
is_using_distance :  True
is_using_contrastive_loss :  True

* Accuracy on training set: 50.00%
* Accuracy on test set: 50.00%
roc_auc_score:  0.50

---------------------------------------
Config:

is_flipped_labels :  True
is_using_densenet :  False
is_using_distance :  True
is_using_contrastive_loss :  True

* Accuracy on training set: 50.00%
* Accuracy on test set: 50.00%
roc_auc_score:  0.22

---------------------------------------
Config:

is_flipped_labels :  False
is_using_densenet :  False
is_using_distance :  True
is_using_contrastive_loss :  True

* Accuracy on training set: 50.00%
* Accuracy on test set: 50.00%
roc_auc_score:  0.50

---------------------------------------
Config:

is_flipped_labels :  False
is_using_densenet :  False
is_using_distance :  False
is_using_contrastive_loss :  True

* Accuracy on training set: 97.86%
* Accuracy on test set: 76.51%
roc_auc_score:  0.90

---------------------------------------
Config:

is_flipped_labels :  False
is_using_densenet :  False
is_using_distance :  True
is_using_contrastive_loss :  False




"""